In [1]:
from dotenv import load_dotenv
import robin_stocks.robinhood as rs
import os
import requests
import time
import json
import smtplib
import math
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pathlib

## Global To-do

1. Setup a ledger for when trades are made with which stock made the trade
        -Need to make sure ledger doesn't overwrite the previous ledger, DONE
2. Check for issues in Stock class
3. update data distribution with clause for if data doesn't come in
4. Inquire/study for a better equation for reserve_cash distribution
5. Need to make the ledger into a .txt file and then attach to email
6. Need to set limit for selling as the sell can now happen anytime to sin goes negative. Need to figure out if I should keep this or not.

## Testing

In [2]:
def create_test_env(top_100):
    test_folder = pathlib.Path("./Test_info")
    file_list = os.listdir(test_folder)
    testing_folder = "./Test_info"
    
    if file_list == []:
        new_file_name = "1000.json"
    else:
        last_file = str(file_list[-1])
        seperated_name = last_file.split(".")
        file_num = seperated_name[0].split("\\")[-1]
        file_name = int(file_num)
        file_name = file_name + 1
        new_file_name = f"{file_name}.json"
        print(file_num)
        print(file_name)
        
    file_path = f"{testing_folder}/{new_file_name}"
    
        
    with open(file_path, "w+") as file:
        json.dump(top_100, file)
        
        
    
    
    

## Initialization and Active Functions
    Functions used when starting and in use

In [3]:
def rh_login():
    load_dotenv()
    robin_user = os.environ.get("robinhood_username")
    robin_pass = os.environ.get("robinhood_password")
    rs.login(username=robin_user,
             password=robin_pass,
             expiresIn=86400,
             by_sms=True)

In [4]:
def time_wizard(time_count):
    time_count += 1
    if time_count == 10:
        time_count = 1
    return time_count

In [5]:
def get_top_100():
    top100_raw_dict = rs.get_top_100()
    top_100_file_path= "./Top_100.json"
    with open(top_100_file_path, "w+") as file:
        json.dump(top100_raw_dict, file)
    return(top100_raw_dict)

In [6]:
def get_money_in_account():
    global reserve_cash
    profile_info = rs.build_user_profile()
    reserve_cash = float(profile_info['cash'])-25000 #to account for buy-in money that cannot be traded
    
    if start_cash is None:
        start_cash = float(profile_info['cash'])-25000

In [7]:
def set_buy_ratio():
    global positive_sin_value
    global reserve_cash
    global start_cash
    value_list = []
    
    for stock_class in positive_sin_value:
        value_list.append(stock_class.sin_factor)
        
    average_sin = sum(value_list) / len(value_list)
    
    for stock_class in positive_sin_value:
        dollar_amount = ((stock_class.sin_factor/average_sin)*reserve_cash)/len(value_list)
        variable_value = (reserve_cash/start_cash)
        if variable_value < .5:
            stock_class.amount_to_spend = dollar_amount
        else:
            stock_class.amount_to_spend = dollar_amount*(variable_value/2)
    
        
            stock_class.sin_factor = dollar_amount
            
    positive_sin_value = []
        
        
    
        

In [8]:
def add_to_log(log_entry):
    log_file_path = "./stock_log.json"
    with open(log_file_path, "a") as file:
        json.dump(log_entry, file)
        file.write('\n')
        

In [9]:
def update_current_time():
    global local_time
    global current_time
    local_time = time.localtime()
    current_time = time.strftime("%H:%M:%S", local_time)
    #hours are in military time
    the_hour = local_time.tm_hour
    the_minutes = local_time.tm_min
    
    if the_hour >= 15 and the_minutes >= 30:
        close_out_accounts()

## Deinitialization
    functions used to close out

In [10]:
def send_ledger_email():
    global start_cash
    global reserve_cash
    load_dotenv()
    my_email = os.environ.get("gmail")
    my_password = os.environ.get("gmail_password")


    sender_email = my_email
    receiver_email = my_email
    email_subject = "Daily Stock Bot Report"
    json_file_path = "SinTest/stock_log.json"
    with open(json_file_path, 'r') as log_file:
        json_contents = json.load(log_file)
    email_message = f"Todays profit/loss: {(reserve_cash - start_cash)}"
        

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = email_subject

    msg.attach(MIMEText(email_message, 'plain'))

    smtp_server = 'smtp.gmail.com'
    smtp_port = 465
    smtp_connection = smtplib.SMTP_SSL(smtp_server, smtp_port)

    smtp_connection.login(my_email, my_password)

    smtp_connection.sendmail(sender_email, receiver_email, msg.as_string())

    smtp_connection.quit()

## Need to do
-Need to clear JSON file at EOD, am not right now for testing

In [11]:
def close_out_accounts():
    global current_time
    global stocks_list
    global reserve_cash
    global start_cash
    
    for symbol, stock in stocks_list.items():
        if stock.shares >= 1:
            stock.sell
        else:
            pass
        
    EOD_ledger_input = f"{current_time}: Closing out for the day. Here is the stocks_lists: {stocks_list}"
    EOD_reserve_cash =f"Final Results; Reserve Cash: {reserve_cash}, Total Profit/Loss: {reserve_cash - start_cash}"
    add_to_log(EOD_ledger_input)
    add_to_log(EOD_reserve_cash)
    
    send_ledger_email()
    
    
    sys.exit()

## Data Distribution
    receives data from get_top_100()
    makes stock classes if no stocks in list
    looks for stock symbol in stock class and will update each class if they are in stocks_list

In [12]:
def update_stock_classes(top100_raw_dict):
    global stocks_list
    dict_symbol_list = []
    
    for stock_dict in top100_raw_dict:
        ssymbol = stock_dict["symbol"]
        dict_symbol_list.append(ssymbol)
        
    for symbol, stock in stocks_list.items():
        if symbol not in dict_symbol_list:
            stock.sell()
            stock.price_history = []
            
    for stock_dict in top100_raw_dict: 
        if stock_dict["symbol"] not in stocks_list:
            stock_class = Stock(stock_dict)
       
        else:
            for stock_name, stock_class in stocks_list.items():
                if stock_name == stock_dict["symbol"]: 
                    stock_class.ask_price = stock_dict.get("ask_price")
                    stock_class.update()
        
    
            
    

## Stock Class
    When class is created all stock data is turned into class variables (Thank you Julian)
    class is added key, value as symbol, class to the stocks_list dictionary
    update method is invoked from update_stock_classes()
    update method includes the equation_of_sin() method    

In [13]:
class Stock:
    #Name
    def __str__(self):
        return self.symbol

    #Enter the Battlefield
    def __init__(self, stock_dict):
        # set the input items as class members
        for key, value in stock_dict.items():
            exec("self." + key + " = value")
            
    #Key Words
        self.stock_dict = stock_dict
        stocks_list[self.symbol] = self #add to dictionary with key as symbol, and value as the class
        self.shares = 0
        self.price_history=[]
        self.sin_factor = None
        self.amount_to_spend = None
        self.price_history.append(self.ask_price)
        self.money_spent = 0.0
        self.loss_tick = 0
        
        
        
    #Abilities
   
    def update(self):
        global time_count
        self.price_history.append(self.ask_price)
        
        #makes the program have at least 5 data points before moving to the next step
        if len(self.price_history) >= 6:
            self.price_history.pop(0)
        else:
            pass
        

        
        if len(self.price_history) >= 5 and time_count == 5:
            self.equation_of_sin()
            
        if self.amount_to_spend is not None:
            self.buy()
            
        
        if self.sin_factor is not None and self.sin_factor < 0 and self.shares != 0:
            self.sell()
        
        
    
    def equation_of_sin(self):
        self.a = len(self.price_history)
        self.price_differential = float(self.price_history[-1]) - float(self.price_history[0])
        self.b = self.price_differential
        self.c = math.sqrt(self.b**2 + self.a**2)
        
        self.sin_factor = round((self.b / self.c), 2)
        
        if self.sin_factor > 0:
            positive_sin_value.append(self)
            
    def buy(self):
        global reserve_cash
        global current_time
        amount_you_can_buy = float(self.amount_to_spend)/float(self.ask_price)
        amount_rounded = round(amount_you_can_buy)
        cost_to_buy= round((float(amount_rounded)*float(self.ask_price)), 2)
        if amount_rounded < 1 or cost_to_buy > reserve_cash:
            pass
        else:
            #print(f"I am buying {amount_rounded} of {self.symbol}")
            reserve_cash = reserve_cash - cost_to_buy
            self.shares += amount_rounded
            self.money_spent += cost_to_buy
           
            buy_log_entry= f"{current_time}: Bought {amount_rounded} shares of {self.symbol} for {cost_to_buy}, Available Funds: {reserve_cash}"
            add_to_log(buy_log_entry)
            
            
        self.amount_to_spend = None
        
        
    def sell(self):
        global reserve_cash
        sell_value = float(self.ask_price)*float(self.shares)
        reserve_cash = reserve_cash + float(sell_value)
        profit_loss = sell_value - self.money_spent
        
        sell_log_entry = f"{current_time}: Sold {self.shares} shares of {self.symbol} for {sell_value}, Available funds: {reserve_cash}, Profit/Loss: {profit_loss}"
        add_to_log(sell_log_entry)
        
        self.shares = 0
        self.money_spent = 0.0

    def print_variable(self):
        print(f"{self.symbol}: {self.sin_factor}")
    

## Staging Area
    Sets Modules
    Sets Global variables
    Keeps time using time_wizard()
    logs into Robinhood using rh_login()
    pulls buying power info from account with get_money_in_account()
    pulls stock data from API using get_top_100()
    stock data goes into update_stock_classes()
    when stocks show positive sin_value, it distributes reserve cash to the stock class for calulating buying

In [14]:
#Pre-Game Actions/Values
rh_login()
time_count = 0
gains_losses = {}
stocks_list = {}
reserve_cash = float(30000)
positive_sin_value = []
local_time = time.localtime()
current_time = time.strftime("%H:%M:%S", local_time)
start_cash = float(30000)

#get_money_in_account()
add_to_log(reserve_cash)


#Main Phase
while True: 
    print(f"TICK: {time_count} \n ----------------------------------------------------------------------------------------------------------------")
    time_count = time_wizard(time_count)
    update_current_time()
    create_test_env(get_top_100())
    update_stock_classes(get_top_100())
    print(positive_sin_value)
    if positive_sin_value != []:
        set_buy_ratio()
    
        
    
    time.sleep(30)

TICK: 0 
 ----------------------------------------------------------------------------------------------------------------
[]
TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
1000
1001
[]
TICK: 2 
 ----------------------------------------------------------------------------------------------------------------
1001
1002
[]
TICK: 3 
 ----------------------------------------------------------------------------------------------------------------
1002
1003
[]
TICK: 4 
 ----------------------------------------------------------------------------------------------------------------
1003
1004
[<__main__.Stock object at 0x0000027E85BE3410>, <__main__.Stock object at 0x0000027E876C0350>, <__main__.Stock object at 0x0000027E876C0610>, <__main__.Stock object at 0x0000027E876C0E10>, <__main__.Stock object at 0x0000027E876C11D0>, <__main__.Stock object at 0x0000027E876C1550>, <__main__.Stock object at 0x0000027E876C18D0>]
TIC

TICK: 3 
 ----------------------------------------------------------------------------------------------------------------
1047
1048
[]
TICK: 4 
 ----------------------------------------------------------------------------------------------------------------
1048
1049
[<__main__.Stock object at 0x0000027E876B8BD0>, <__main__.Stock object at 0x0000027E85BE3410>, <__main__.Stock object at 0x0000027E876C0350>, <__main__.Stock object at 0x0000027E876C08D0>, <__main__.Stock object at 0x0000027E876C0910>, <__main__.Stock object at 0x0000027E876C0AD0>, <__main__.Stock object at 0x0000027E876C1D50>, <__main__.Stock object at 0x0000027E876C2A90>, <__main__.Stock object at 0x0000027E876F9390>, <__main__.Stock object at 0x0000027E876F9410>]
TICK: 5 
 ----------------------------------------------------------------------------------------------------------------
1049
1050
[]
TICK: 6 
 ----------------------------------------------------------------------------------------------------------------
1

[]
TICK: 7 
 ----------------------------------------------------------------------------------------------------------------
1087
1088
[]
TICK: 8 
 ----------------------------------------------------------------------------------------------------------------
1088
1089
[]
TICK: 9 
 ----------------------------------------------------------------------------------------------------------------
1089
1090
[]
TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
1090
1091
[]
TICK: 2 
 ----------------------------------------------------------------------------------------------------------------
1091
1092
[]
TICK: 3 
 ----------------------------------------------------------------------------------------------------------------
1092
1093
[]
TICK: 4 
 ----------------------------------------------------------------------------------------------------------------
1093
1094
[<__main__.Stock object at 0x0000027E876C1DD0>, 

[]
TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
1135
1136
[]
TICK: 2 
 ----------------------------------------------------------------------------------------------------------------
1136
1137
[]
TICK: 3 
 ----------------------------------------------------------------------------------------------------------------
1137
1138
[]
TICK: 4 
 ----------------------------------------------------------------------------------------------------------------
1138
1139
[<__main__.Stock object at 0x0000027E876C0910>, <__main__.Stock object at 0x0000027E876C2610>]
TICK: 5 
 ----------------------------------------------------------------------------------------------------------------
1139
1140
[]
TICK: 6 
 ----------------------------------------------------------------------------------------------------------------
1140
1141
[]
TICK: 7 
 -------------------------------------------------------------------------------

TICK: 7 
 ----------------------------------------------------------------------------------------------------------------
1177
1178
[]
TICK: 8 
 ----------------------------------------------------------------------------------------------------------------
1178
1179
[]
TICK: 9 
 ----------------------------------------------------------------------------------------------------------------
1179
1180
[]
TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
1180
1181
[]
TICK: 2 
 ----------------------------------------------------------------------------------------------------------------
1181
1182
[]
TICK: 3 
 ----------------------------------------------------------------------------------------------------------------
1182
1183
[]
TICK: 4 
 ----------------------------------------------------------------------------------------------------------------
1183
1184
[<__main__.Stock object at 0x0000027E876C0CD0>, <__

...

TICK: 5 
 ----------------------------------------------------------------------------------------------------------------
1958
1959
[]
TICK: 6 
 ----------------------------------------------------------------------------------------------------------------
1959
1960
[]
TICK: 7 
 ----------------------------------------------------------------------------------------------------------------
1960
1961
[]
TICK: 8 
 ----------------------------------------------------------------------------------------------------------------
1961
1962
[]
TICK: 9 
 ----------------------------------------------------------------------------------------------------------------
1962
1963
[]
TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
1963
1964
[]
TICK: 2 
 ----------------------------------------------------------------------------------------------------------------
1964
1965
[]
TICK: 3 
 --------------------------------------

TICK: 5 
 ----------------------------------------------------------------------------------------------------------------
1994
1995
[]
TICK: 6 
 ----------------------------------------------------------------------------------------------------------------
1995
1996
[]
TICK: 7 
 ----------------------------------------------------------------------------------------------------------------
1996
1997
[]
TICK: 8 
 ----------------------------------------------------------------------------------------------------------------
1997
1998
[]
TICK: 9 
 ----------------------------------------------------------------------------------------------------------------
1998
1999
[]
TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
1999
2000
[]
TICK: 2 
 ----------------------------------------------------------------------------------------------------------------
2000
2001
[]
TICK: 3 
 --------------------------------------

TICK: 1 
 ----------------------------------------------------------------------------------------------------------------
2026
2027
[]
TICK: 2 
 ----------------------------------------------------------------------------------------------------------------
2027
2028
[]
TICK: 3 
 ----------------------------------------------------------------------------------------------------------------
2028
2029
[]
TICK: 4 
 ----------------------------------------------------------------------------------------------------------------
2029
2030
[<__main__.Stock object at 0x0000027E876B8BD0>, <__main__.Stock object at 0x0000027E876C04D0>, <__main__.Stock object at 0x0000027E876F9290>, <__main__.Stock object at 0x0000027E876F9ED0>, <__main__.Stock object at 0x0000027E876F9F90>, <__main__.Stock object at 0x0000027E876FA1D0>, <__main__.Stock object at 0x0000027E876FA290>]
TICK: 5 
 ----------------------------------------------------------------------------------------------------------------
2030
2

TICK: 5 
 ----------------------------------------------------------------------------------------------------------------
2066
2067
[]
TICK: 6 
 ----------------------------------------------------------------------------------------------------------------
2067
2068
[]
TICK: 7 
 ----------------------------------------------------------------------------------------------------------------
2068
2069
[]
TICK: 8 
 ----------------------------------------------------------------------------------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: 'SinTest/stock_log.json'

##  Dictionary/List Reminders
positive_sin_value = classes with positive sin value

stocks_list (x:y), x= stock symbol, y=associated stock class

stockdict, check Top_100.json

## This to change before Release
1. set reserve_cash to pull from the server, not be set like for testing
        -need to choose between pulling reserve cash only once at beginning of day or possibly repulling after a successful sell to basically do a doublecheck
        
2.Need to set actual buy and sell methods to interact with RH



## Equation for dividing available money between available stocks
s = personal stock sin
a = average stock sin
m = money in pool
n = number of stocks with a positive sin

(((s/a)*m)/n)s

Look up CMP magic method for comparing stock classes 
Look up __setattr__(self, name, value) for stock classes instead of exec function
Self-study: sets